In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as _sum

In [2]:
conf = SparkConf()
conf.set("spark.hadoop.fs.s3a.impl",
                 "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.access.key", "admin")
conf.set("spark.hadoop.fs.s3a.secret.key", "123456789")
conf.set("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider',
            'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set(
    "spark.jars.packages",
    "io.delta:delta-core_2.12:1.0.0,"
    "org.apache.hadoop:hadoop-aws:3.1.1,"
    "com.amazonaws:aws-java-sdk:1.11.271,"
    "com.amazonaws:aws-java-sdk-bundle:1.11.271,"
    "software.amazon.awssdk:url-connection-client:2.15.40",
)
conf.set("spark.sql.extensions",
            "io.delta.sql.DeltaSparkSessionExtension")
conf.set(
    "spark.sql.catalog.spark_catalog",
    "org.apache.spark.sql.delta.catalog.DeltaCatalog",
)
conf.set("spark.databricks.delta.merge.repartitionBeforeWrite.enabled","true")

spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .master("local[*]")
    .getOrCreate()
)

23/04/18 23:04:36 WARN Utils: Your hostname, ducdn-G3-3579 resolves to a loopback address: 127.0.1.1; using 192.168.2.106 instead (on interface wlo1)
23/04/18 23:04:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ducdn/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ducdn/.ivy2/cache
The jars for the packages stored in: /home/ducdn/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-75e39b92-49ad-413f-bd2b-7743896b445e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.hadoop#hadoop-aws;3.1.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central

### Bronzen

#### Orders

In [3]:
orderDF = spark.read.format("delta").load("s3a://datalake/brozen/cdc.myshop.orders")
orderDF.show()

23/04/18 23:04:59 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---+-------------+---+-------+-----------+---------+----------+---------+----+-----+---+
| op|        ts_ms| id|user_id|    payment|status_id|created_at|before_id|year|month|day|
+---+-------------+---+-------+-----------+---------+----------+---------+----+-----+---+
|  c|1681833006820| 57|    577|credit_card|        2|1675006844|       57|2023|    1| 29|
|  c|1681832645426| 21|   6733|credit_card|        1|1679326844|       21|2023|    3| 20|
|  c|1681832866321| 43|   6889|credit_card|        3|1681659644|       43|2023|    4| 16|
|  c|1681832705669| 27|   8965|credit_card|        3|1656258044|       27|2022|    6| 26|
|  c|1681832524949|  9|   3370|credit_card|        2|1678117244|        9|2023|    3|  6|
|  c|1681833127311| 69|   1264|credit_card|        1|1678462844|       69|2023|    3| 10|
|  r|1681832458594|  1|   8894|credit_card|        4|1659195644|        1|2022|    7| 30|
|  c|1681833077061| 64|   9250|credit_card|        1|1661960444|       64|2022|    8| 31|
|  c|16818

#### Products

#### Users

In [5]:
users = spark.read.format("delta").load("s3a://datalake/brozen/cdc.myshop.users")
users.count()

9999

In [14]:
users.createOrReplaceTempView("users")

spark.sql("select count(*) from users").show()

+--------+
|count(1)|
+--------+
|    9998|
+--------+



#### order detail

In [22]:
order_detail = spark.read.format("delta").load("s3a://datalake/brozen/cdc.myshop.order_detail")
order_detail.createOrReplaceTempView("order_detail")
spark.sql("select * from order_detail where order_id = 1").show()

+---+-------------+--------+----------+--------+-----------+----------+---------------+-----------------+
| op|        ts_ms|order_id|product_id|quantity|total_price| create_at|before_order_id|before_product_id|
+---+-------------+--------+----------+--------+-----------+----------+---------------+-----------------+
|  c|1680506120979|       1|       224|       6|     7626.0|1680506120|              1|              224|
|  c|1680506120980|       1|       544|       3|     5202.0|1680506120|              1|              544|
+---+-------------+--------+----------+--------+-----------+----------+---------------+-----------------+



In [21]:
order_detail = spark.read.format("delta").load("s3a://datalake/brozen/cdc.myshop.order_detail")
order_detail.createOrReplaceTempView("order_detail")
spark.sql("select * from order_detail").show()

+---+-------------+--------+----------+--------+-----------+----------+---------------+-----------------+
| op|        ts_ms|order_id|product_id|quantity|total_price| create_at|before_order_id|before_product_id|
+---+-------------+--------+----------+--------+-----------+----------+---------------+-----------------+
|  c|1680508336597|       7|       426|       2|     1102.0|1680508306|              7|              426|
|  c|1680508376686|      11|       732|       1|     1673.0|1680508306|             11|              732|
|  c|1680513026109|      21|       643|       5|     5210.0|1680513026|             21|              643|
|  c|1680508376686|      11|       209|       2|      548.0|1680508306|             11|              209|
|  c|1680508386713|      12|       132|       1|      447.0|1680508306|             12|              132|
|  c|1680508306502|       4|       164|       4|     7036.0|1680508306|              4|              164|
|  c|1680508326562|       6|       755|       

### Sliver

#### order detail

In [5]:
order_detail = spark.read.format("delta").load("s3a://datalake/sliver/cdc.myshop.order_detail")
order_detail.orderBy("order_id").show()

+---+-------------+--------+----------+--------+-----------+----------+--------------------+----+-----+---+
| op|        ts_ms|order_id|product_id|quantity|total_price| create_at|           timestamp|year|month|day|
+---+-------------+--------+----------+--------+-----------+----------+--------------------+----+-----+---+
|  c|1680532707945|       1|       160|       6|     9348.0|1680532707|2023-04-03 21:38:...|2023|    4|  3|
|  c|1680532707945|       1|       634|       6|     5070.0|1680532707|2023-04-03 21:38:...|2023|    4|  3|
|  c|1680532717983|       2|        28|       6|    10464.0|1680532707|2023-04-03 21:38:...|2023|    4|  3|
|  c|1680532717983|       2|       457|       4|      712.0|1680532707|2023-04-03 21:38:...|2023|    4|  3|
|  c|1680532717983|       2|       597|       5|     1195.0|1680532707|2023-04-03 21:38:...|2023|    4|  3|
|  c|1680532717983|       2|       137|       4|      952.0|1680532707|2023-04-03 21:38:...|2023|    4|  3|
|  c|1680532717983|       2|

In [14]:
orders = spark.read.format("delta").load("s3a://datalake/sliver/cdc.myshop.order_detail")
orders.count()

41

In [ ]:
path_table = "s3a://datalake/brozen/cdc.inventory.orders"
df = spark.read.format("delta").option("versionAsOf", 0).load(path_table)
df.show()

In [ ]:
delta_table = DeltaTable.forPath(spark, "s3a://datalake/brozen/cdc.inventory.orders")
history = delta_table.history()
history.show()

In [ ]:
df = spark.read.format("delta").load("s3a://datalake/sliver")
df.show()